# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,2.80,91,95,2.57,SJ,1721237499
1,1,ilulissat,69.2167,-51.1000,13.01,47,0,1.54,GL,1721237501
2,2,vorgashor,67.5833,63.9500,15.66,77,87,0.19,RU,1721237502
3,3,waitangi,-43.9535,-176.5597,9.34,91,99,1.34,NZ,1721237503
4,4,tobelo,1.7284,128.0095,25.83,85,98,3.79,ID,1721237504


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'EsriNatGeo',
    size = 'Humidity',
    scale = 1,
    color = 'City',
    hover_cols = 'City'
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_filtered = city_data_df[(city_data_df["Max Temp"] < 80) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_weather_df = city_data_df_filtered.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,ilulissat,69.2167,-51.1000,13.01,47,0,1.54,GL,1721237501
5,5,alice springs,-23.7000,133.8833,1.75,87,0,2.06,AU,1721237505
10,10,corinto,37.9407,22.9573,34.94,36,0,2.00,GR,1721237512
25,25,ushuaia,-54.8000,-68.3000,7.81,71,0,2.57,AR,1721237530
34,34,unai,-16.3575,-46.9061,29.03,28,0,1.79,BR,1721237587
...,...,...,...,...,...,...,...,...,...,...
555,555,ihosy,-22.4000,46.1167,16.27,42,0,1.34,MG,1721238224
577,577,maputo,-25.9653,32.5892,19.82,77,0,3.09,MZ,1721238255
580,580,ardestan,33.3761,52.3694,33.20,15,0,4.34,IR,1721238259
583,583,talara,-4.5772,-81.2719,24.42,64,0,9.37,PE,1721238262


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,ilulissat,GL,69.2167,-51.1000,47,
5,alice springs,AU,-23.7000,133.8833,87,
10,corinto,GR,37.9407,22.9573,36,
25,ushuaia,AR,-54.8000,-68.3000,71,
34,unai,BR,-16.3575,-46.9061,28,
...,...,...,...,...,...,...
555,ihosy,MG,-22.4000,46.1167,42,
577,maputo,MZ,-25.9653,32.5892,77,
580,ardestan,IR,33.3761,52.3694,15,
583,talara,PE,-4.5772,-81.2719,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng}, {lat}, {radius}"
    params["bias"] = f"proximity:{lng}, {lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ilulissat - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
corinto - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
unai - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
sheridan - nearest hotel: No hotel found
longyearbyen - nearest hotel: No hotel found
shanhecun - nearest hotel: No hotel found
aripuana - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found


KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)